In [1]:
"""
Importing python packages and and functions.  
"""

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw
from Diffusion_Functions import frame_adjust, MSD_Deff_Calcs, prettify2, fillin2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [63]:
"""
This block is the driver for the rest of the code.  With a few exceptions (for the time being) the user only needs
to make changes in this block in order to run the code.
""" 

# Change the variable 'name' to change file names of resulting images.
name = 'confocal_625ms_60x'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)
Tplot = '{}_Tplot'.format(name)


# "Conversion" contains the necessary conversions from the impages.  First, microns per pixel for xy images (umppx),
# second, frames per second (fps), and third, mcirons per slice for z video (set to 1 when using 2D data).
conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.83, 1.62, 1)
conversion[2] = (0.83, 1.62, 1)
conversion[3] = (0.83, 1.62, 1)


# The following variables are used to filter out trajectories.  'Cut' is the minimum number of frames a trajectory
# must have to be included in the analysis.  'Great' is the maximum MSD a particle can move in the video to be in-
# cluded in the analysis.  'Filtered' turns the ability to manually filter particles on and off.  If 'filtered' is on,
# then the user must enter the particle numbers in the array 'tofilt.'

cut = 5
great = 1
filtered = True
new_method = True
tofilt = np.array([11,  80,  95, 123, 124, 130, 140, 141, 178, 180, 182, 183, 185,
       187, 188, 190, 192, 195, 197, 201, 202, 206, 210, 211, 215, 216,
       217, 219, 221, 223, 224, 225, 229, 230, 234, 235, 237, 247, 276,
       282, 298, 308, 309, 310, 318, 319, 323, 329, 365, 381, 408, 414,
       418, 428, 429, 432, 436, 437, 444, 449, 450, 451, 453, 454, 465,
       467, 468, 471, 473, 474, 480, 506, 509, 510, 514, 515, 516, 531,
       647, 650, 651, 654, 657, 665, 669, 671, 673, 674, 676, 678, 679,
       683, 685, 688, 689, 690, 692, 694, 697, 699, 700])


# The user enters the names of the csv files to be included in the analysis here.  The user must also specify the 
# number of videos in the analysis in 'totvids.'
trajectory = dict()
totvids = 3

trajectory[1] = np.genfromtxt('./confocal_625ms_60x_1.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('./confocal_625ms_60x_2.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('./confocal_625ms_60x_3.csv',
            delimiter =",")



for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)


# Collect trajectories of separate videos into one array and updates the numbering.
# Comment this section out to make useable for a single trajectory.
parts = dict()
tots = dict()
newtots = dict()
newtots[0] = 0
tlen = dict()
tlength = dict()
tlength[0] = 0

for num in range(1, totvids + 1):
    tots[num] = trajectory[num][-1, 0].astype(np.int64)
    parts[num] = tots[num]
    counter = 1
    newtots[num] = newtots[num-1] + tots[num]
    
    tlen[num] = trajectory[num].shape[0]
    tlength[num] = tlength[num-1] + tlen[num]


placeholder = np.zeros((tlength[totvids], 11))

for num in range(1, totvids + 1):
    placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
    placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

# trajectory[1] = placeholder

# trajectory[2][:,0] = trajectory[2][:,0] + tots[1]
# placeholder = np.append(placeholder, trajectory[2], axis=0)
# trajectory[3][:,0] = trajectory[3][:,0] + tots[2]
# placeholder = np.append(placeholder, trajectory[3], axis=0)

In [3]:
trajectory[3].shape

(12395, 11)

In [4]:
print('Total # of particles in each video', newtots)

Total # of particles in each video {0: 0, 1: 317, 2: 637, 3: 912}


In [5]:
# Theoretical diffusion calculation using Stokes-Einstein.  Required inputs: TC (Temperature in Celsius), mu1 and mu2
# (viscosities at two temperatures to perform linear regression), TC1 and TC2 (Temperatures at which mu1 and mu2 were
# taken). and radius (radius of particles in m).

TC = 37
mu1 = 1.002*10**-3; mu2 = 0.798*10**-3; TC1 = 20; TC2 = 30
radius = (128/2) * 10**-9 #m

k = 1.380649 * 10**-23
TK = TC + 273
mu = (mu2 - mu1)*(TC - TC1)/(TC2 - TC1) + mu1

Deff1 = k*TK/(6*np.pi*mu*radius) #m2/s
Deff2 = Deff1 * 10**12 #um2/s

print('Theoretical diffusion coefficient', np.round(Deff2, 4), 'um2/s')

Theoretical diffusion coefficient 5.4149 um2/s


In [6]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.
    
    Input: trajectory dataset from MOSAIC tracking software read into a numpy array
    Output: modified numpy array with missing frames filled in.
    """

    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [7]:
# Creates 'time' and 'frames' variables to keep track of time and frames for all xyz, MSD, and diffusion datasets.
# There is a single 'time' and 'frames' variable rather than having one for each particle.  This works because all the
# blanks in 'trajectory' were filled in.

rawframes = placeholder[:, 1]
#frames = np.linspace()
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]


# Creates dictionaries to save all future xyz and MSD datasets.

x = dict() #xyz trajectory data with particle numbers as keys.  
y = dict()
z = dict()

xs = dict() #shifted xyz trajectory data such that the frame in which the particle appears is the first frame.
ys = dict()
zs = dict()

M1x = dict() # MSD dictionaries (without shifting)
M1y = dict()
M1z = dict()
M2xy = dict()
M2xz = dict()
M2yz = dict()
M3 = dict()

SM1x = dict() # Shifted MSD dictionaries.
SM1y = dict()
SM1z = dict()
SM2xy = dict()
SM2xz = dict()
SM2yz = dict()
SM3 = dict()

SD1x = dict() # Shifted diffusion coefficient dictionaries.
SD1y = dict()
SD1z = dict()
SD2xy = dict()
SD2xz = dict()
SD2yz = dict()
SD3 = dict()

In [8]:
print('Total number of frames', time.shape[0])

Total number of frames 74


In [9]:
"""
This block creates a new variable 'fixed' that performs the necessary conversions on the xyz data to convert from 
pixels to microns.
"""

dataset = dict()

rawdataset = np.zeros(placeholder.shape)
particles = placeholder[:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = placeholder[:, :]

# Old code that had errors in it.  Every time the code was re-run, the conversion factors were re-applied.
#Perform conversion on raw dataset
# rawdataset[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
# rawdataset[:, 4] = conversion[1][2] * rawdataset[:, 4]

fixed = np.zeros(placeholder.shape)
fixed[:, 0:2] = rawdataset[:, 0:2]
fixed[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
fixed[:, 4] = conversion[1][2] * rawdataset[:, 4]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (fixed[min1:max1+1, 0:5])

# Performs a check on the output of dataset.  Sees if frames start at 0, and if xyz data is present.
# dataset[540]

In [10]:
"""
This block fills in the xyz and shifted xyz dictionaries.
It also creates a useful variable, 'I.'  Ultimately, it will keep track of the first non-zero element in each
trajectory.
"""

I = dict()
for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
    z[num] = np.zeros(frames.shape[0])
    z[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 4]
    
    xs[num] = np.zeros(frames.shape[0])
    xs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 2]
    ys[num] = np.zeros(frames.shape[0])
    ys[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 3]
    zs[num] = np.zeros(frames.shape[0])
    zs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 4]

In [11]:
"""
This block filters out trajectories that don't have the minimum number of frames specified by the variable 'cut.'
""" 

cutoff = cut
x1 = dict()
y1 = dict()
z1 = dict()

xs1 = dict()
ys1 = dict()
zs1 = dict()

fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]
        z1[num - nones] = z[num]
        
        xs1[num - nones] = xs[num]
        ys1[num - nones] = ys[num]
        zs1[num - nones] = zs[num]
        # I must also redefine the particle numbers to reflect the new set.
        #teancum[num - nones][:, 0] = fifties

x = x1
y = y1
z = z1

xs = xs1
ys = ys1
zs = zs1

for num in range(1, fifties):
    xs[num] = ma.masked_equal(xs[num], 0)
    ys[num] = ma.masked_equal(ys[num], 0)
    zs[num] = ma.masked_equal(zs[num], 0)

    x[num] = ma.masked_equal(x[num], 0)
    y[num] = ma.masked_equal(y[num], 0)
    z[num] = ma.masked_equal(z[num], 0)

In [12]:
#fifties is the new total1 after filtering out short trajectories
total1 = fifties + 1

print('Total particles after merging datasets and filtering short trajectories:', fifties)

Total particles after merging datasets and filtering short trajectories: 700


In [13]:
"""
This block fills in the MSD dictionaries using the old method (a fixed starting point and a single lag time).  In the
future, I will create an if statement that will allow the user to toggle between the old method and new method. The 
new method is already present at the end of this notebook, but I need to implement it throughout.
"""

xymask = dict()

# Intermediates for new method
m1xa = dict()
m1ya = dict()
m1za = dict()
m2xya = dict()
m2xza = dict()
m2yza = dict()
m3a = dict()

#numpy array containing all frames of all particles using lag MSD method
a1x = np.zeros((total1, frames.shape[0]))
a1y = np.zeros((total1, frames.shape[0]))
a1z = np.zeros((total1, frames.shape[0]))
a2xy = np.zeros((total1, frames.shape[0]))
a2xz = np.zeros((total1, frames.shape[0]))
a2yz = np.zeros((total1, frames.shape[0]))
a3 = np.zeros((total1, frames.shape[0]))

for num in range(1, total1):

    #Construct MSD and Deff dictionaries
    M1x[num] = np.zeros(frames.shape[0])
    M1x[num][0:frames.shape[0]] = 0
    M1y[num] = np.zeros(frames.shape[0])
    M1y[num][0:frames.shape[0]] = 0
    M1z[num] = np.zeros(frames.shape[0])
    M1z[num][0:frames.shape[0]] = 0
    M2xy[num] = np.zeros(frames.shape[0])
    M2xy[num][0:frames.shape[0]] = 0
    M2xz[num] = np.zeros(frames.shape[0])
    M2xz[num][0:frames.shape[0]] = 0
    M2yz[num] = np.zeros(frames.shape[0])
    M2yz[num][0:frames.shape[0]] = 0
    M3[num] = np.zeros(frames.shape[0])
    M3[num][0:frames.shape[0]] = 0
    
    SM1x[num] = np.zeros(frames.shape[0])
    SM1y[num] = np.zeros(frames.shape[0])
    SM1z[num] = np.zeros(frames.shape[0])
    SM2xy[num] = np.zeros(frames.shape[0])
    SM2xz[num] = np.zeros(frames.shape[0])
    SM2yz[num] = np.zeros(frames.shape[0])
    SM3[num] = np.zeros(frames.shape[0])
    
    SD1x[num] = np.zeros(frames.shape[0])
    SD1y[num] = np.zeros(frames.shape[0])
    SD1z[num] = np.zeros(frames.shape[0])
    SD2xy[num] = np.zeros(frames.shape[0])
    SD2xz[num] = np.zeros(frames.shape[0])
    SD2yz[num] = np.zeros(frames.shape[0])
    SD3[num] = np.zeros(frames.shape[0])
    
    #Fill in MSD and Deff dictionaries
    I[num] = np.nonzero(x[num])[0]
    first = I[num][0]
    last = I[num][-1] + 1
    startx = x[num][first]
    starty = y[num][first]
    startz = z[num][first]
    
    if new_method == True:
        
        m1xa[num] = dict()
        m1ya[num] = dict()
        m1za[num] = dict()
        m2xya[num] = dict()
        m2xza[num] = dict()
        m2yza[num] = dict()
        m3a[num] = dict()
        
        for num1 in range(1, frames.shape[0]):

            tau = num1
        
            m1xa[num][num1] = np.zeros(frames.shape[0] - tau)
            m1ya[num][num1] = np.zeros(frames.shape[0] - tau)
            m1za[num][num1] = np.zeros(frames.shape[0] - tau)
            m2xya[num][num1] = np.zeros(frames.shape[0] - tau)
            m2xza[num][num1] = np.zeros(frames.shape[0] - tau)
            m2yza[num][num1] = np.zeros(frames.shape[0] - tau)
            m3a[num][num1] = np.zeros(frames.shape[0] - tau)

            for num2 in range(0, frames.shape[0] - tau):
                m1xa[num][num1][num2] = (xs[num][num2 + tau] - xs[num][num2])**2
                m1ya[num][num1][num2] = (ys[num][num2 + tau] - ys[num][num2])**2
                m1za[num][num1][num2] = (zs[num][num2 + tau] - zs[num][num2])**2
                m2xya[num][num1][num2] = m1xa[num][num1][num2] + m1ya[num][num1][num2]
                m2xza[num][num1][num2] = m1xa[num][num1][num2] + m1za[num][num1][num2]
                m2yza[num][num1][num2] = m1za[num][num1][num2] + m1ya[num][num1][num2]
                m3a[num][num1][num2] = m1xa[num][num1][num2] + m1ya[num][num1][num2] + m1za[num][num1][num2]
                
            m1xa[num][num1] = ma.masked_invalid(m1xa[num][num1])
            m1ya[num][num1] = ma.masked_invalid(m1ya[num][num1])
            m1za[num][num1] = ma.masked_invalid(m1za[num][num1])
            m2xya[num][num1] = ma.masked_invalid(m2xya[num][num1])
            m2xza[num][num1] = ma.masked_invalid(m2xza[num][num1])
            m2yza[num][num1] = ma.masked_invalid(m2yza[num][num1])
            m3a[num][num1] = ma.masked_invalid(m3a[num][num1])

            SM1x[num][num1] = np.mean(m1xa[num][num1])
            SM1y[num][num1] = np.mean(m1ya[num][num1])
            SM1z[num][num1] = np.mean(m1za[num][num1])
            SM2xy[num][num1] = np.mean(m2xya[num][num1])
            SM2xz[num][num1] = np.mean(m2xza[num][num1])
            SM2yz[num][num1] = np.mean(m2yza[num][num1])
            SM3[num][num1] = np.mean(m3a[num][num1])

        SM1x[num] = ma.masked_invalid(SM1x[num])
        SM1y[num] = ma.masked_invalid(SM1y[num])
        SM1z[num] = ma.masked_invalid(SM1z[num])
        SM2xy[num] = ma.masked_invalid(SM2xy[num])
        SM2xz[num] = ma.masked_invalid(SM2xz[num])
        SM2yz[num] = ma.masked_invalid(SM2yz[num])
        SM3[num] = ma.masked_invalid(SM3[num])

        M1x[num][first:] = SM1x[num][:frames.shape[0]-first]
        M1y[num][first:] = SM1y[num][:frames.shape[0]-first]
        M1z[num][first:] = SM1z[num][:frames.shape[0]-first]
        M2xy[num][first:] = SM2xy[num][:frames.shape[0]-first]
        M2xz[num][first:] = SM2xz[num][:frames.shape[0]-first]
        M2yz[num][first:] = SM2yz[num][:frames.shape[0]-first]
        M3[num][first:] = SM3[num][:frames.shape[0]-first]
        
        M1x[num] = ma.masked_invalid(M1x[num])
        M1y[num] = ma.masked_invalid(M1y[num])
        M1z[num] = ma.masked_invalid(M1z[num])
        M2xy[num] = ma.masked_invalid(M2xy[num])
        M2xz[num] = ma.masked_invalid(M2xz[num])
        M2yz[num] = ma.masked_invalid(M2yz[num])
        M3[num] = ma.masked_invalid(M3[num])
        
    else:  

        for num1 in range(first, last):
            M1x[num][num1] = (x[num][num1] - startx)**2
            M1y[num][num1] = (y[num][num1] - starty)**2
            M1z[num][num1] = (z[num][num1] - startz)**2
            M2xy[num][num1] = (x[num][num1] - startx)**2 + (y[num][num1] - starty)**2
            M2xz[num][num1] = (x[num][num1] - startx)**2 + (z[num][num1] - startz)**2
            M2yz[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2
            M3[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2 + (x[num][num1] - startx)**2

    # Masks MSD entries that are zero and those that are too large (specified by 'great')
    M1x[num] = ma.masked_equal(M1x[num], 0)
    M1y[num] = ma.masked_equal(M1y[num], 0)
    M1z[num] = ma.masked_equal(M1z[num], 0)
    M2xy[num] = ma.masked_equal(M2xy[num], 0)
    M2xz[num] = ma.masked_equal(M2xz[num], 0)
    M2yz[num] = ma.masked_equal(M2yz[num], 0)
    M3[num] = ma.masked_equal(M3[num], 0)
    
#     M1x[num] = ma.masked_greater(M1x[num], great)
#     M1y[num] = ma.masked_greater(M1y[num], great)
#     M1z[num] = ma.masked_greater(M1z[num], great)
#     M2xy[num] = ma.masked_greater(M2xy[num], great)
#     M2xz[num] = ma.masked_greater(M2xz[num], great)
#     M2yz[num] = ma.masked_greater(M2yz[num], great)
#     M3[num] = ma.masked_greater(M3[num], great)
    
    if new_method == False:
        #Make shifted MSD arrays for geometric and arithmetic averages.

        if M2xy[num][~M2xy[num].mask].shape[0] != 0:
            fir = np.nonzero(M2xy[num])[0][0] - 1
            firl = M1x[num][fir:].shape[0]
        else:
            fir = 0
            firl = 46

        SM1x[num][0:firl] = M1x[num][fir:] #Fills in shifted MSD dictionaries
        SM1y[num][0:firl] = M1y[num][fir:]
        SM1z[num][0:firl] = M1z[num][fir:]
        SM2xy[num][0:firl] = M2xy[num][fir:]
        SM2xz[num][0:firl] = M2xz[num][fir:]
        SM2yz[num][0:firl] = M2yz[num][fir:]
        SM3[num][0:firl] = M3[num][fir:]

#     SM1x[num] = ma.masked_greater(SM1x[num], great) # Masks entries with MSDs that are too large.
#     SM1y[num] = ma.masked_greater(SM1y[num], great)
#     SM1z[num] = ma.masked_greater(SM1z[num], great)
#     SM2xy[num] = ma.masked_greater(SM2xy[num], great)
#     SM2xz[num] = ma.masked_greater(SM2xz[num], great)
#     SM2yz[num] = ma.masked_greater(SM2yz[num], great)
#     SM3[num] = ma.masked_greater(SM3[num], great)

    SM1x[num] = ma.masked_equal(SM1x[num], 0)
    SM1y[num] = ma.masked_equal(SM1y[num], 0)
    SM1z[num] = ma.masked_equal(SM1z[num], 0)
    SM2xy[num] = ma.masked_equal(SM2xy[num], 0)
    SM2xz[num] = ma.masked_equal(SM2xz[num], 0)
    SM2yz[num] = ma.masked_equal(SM2yz[num], 0)
    SM3[num] = ma.masked_equal(SM3[num], 0)

    # Applies masks from shifted MSDs to shifted xyz trajectories as well.  Important, as I use those
    # updated trajectories to calculate MSDs using the new method at the bottom.
    xymask[num] = SM2xy[num].recordmask
    xs[num] = ma.array(xs[num], mask = xymask[num])
    ys[num] = ma.array(ys[num], mask = xymask[num])
    zs[num] = ma.array(zs[num], mask = xymask[num])
    
    xs[num] = ma.masked_equal(xs[num], 0)
    ys[num] = ma.masked_equal(ys[num], 0)
    zs[num] = ma.masked_equal(zs[num], 0)
    
    
    SD1x[num][1:] = SM1x[num][1:]/(2*time[1:]) # Calculating diffusion coefficients.
    SD1y[num][1:] = SM1y[num][1:]/(2*time[1:])
    SD1z[num][1:] = SM1z[num][1:]/(2*time[1:])
    SD2xy[num][1:] = SM2xy[num][1:]/(4*time[1:])
    SD2xz[num][1:] = SM2xz[num][1:]/(4*time[1:])
    SD2yz[num][1:] = SM2yz[num][1:]/(4*time[1:])
    SD3[num][1:] = SM3[num][1:]/(6*time[1:])

    SD1x[num] = ma.masked_equal(SD1x[num], 0)
    SD1y[num] = ma.masked_equal(SD1y[num], 0)
    SD1z[num] = ma.masked_equal(SD1z[num], 0)
    SD2xy[num] = ma.masked_equal(SD2xy[num], 0)
    SD2xz[num] = ma.masked_equal(SD2xz[num], 0)
    SD2yz[num] = ma.masked_equal(SD2yz[num], 0)
    SD3[num] = ma.masked_equal(SD3[num], 0)
    
    a1x[num-1, :] = SM1x[num]
    a1y[num-1, :] = SM1y[num]
    a1z[num-1, :] = SM1z[num]
    a2xy[num-1, :] = SM2xy[num]
    a2xz[num-1, :] = SM2xz[num]
    a2yz[num-1, :] = SM2yz[num]
    a3[num-1, :] = SM3[num]

    a1x[num] = ma.masked_equal(a1x[num], 0)
    a1y[num] = ma.masked_equal(a1y[num], 0)
    a1z[num] = ma.masked_equal(a1z[num], 0)
    a2xy[num] = ma.masked_equal(a2xy[num], 0)
    a2xz[num] = ma.masked_equal(a2xz[num], 0)
    a2yz[num] = ma.masked_equal(a2yz[num], 0)
    a3[num] = ma.masked_equal(a3[num], 0)

    a1x[num] = ma.masked_invalid(a1x[num])
    a1y[num] = ma.masked_invalid(a1y[num])
    a1z[num] = ma.masked_invalid(a1z[num])
    a2xy[num] = ma.masked_invalid(a2xy[num])
    a2xz[num] = ma.masked_invalid(a2xz[num])
    a2yz[num] = ma.masked_invalid(a2yz[num])
    a3[num] = ma.masked_invalid(a3[num])

#     a1x[num] = ma.masked_greater(a1x[num], great) # Masks entries with MSDs that are too large.
#     a1y[num] = ma.masked_greater(a1y[num], great)
#     a1z[num] = ma.masked_greater(a1z[num], great)
#     a2xy[num] = ma.masked_greater(a2xy[num], great)
#     a2xz[num] = ma.masked_greater(a2xz[num], great)
#     a2yz[num] = ma.masked_greater(a2yz[num], great)
#     a3[num] = ma.masked_greater(a3[num], great)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [14]:
num = 200
tau = 1
num2 = 6

(xs[num][num2 + tau] - xs[num][num2])**2

0.0

In [15]:
"""
This block is a final filter again large datasets.  It checks to see if there are any MSDs greater than a certain
threshold at a given tau.
"""

thresh1 = great
given_tau = 5

filte = np.where(a2xy[:, given_tau]> thresh1)[0]+1
filte

array([ 11,  80,  95, 123, 124, 130, 140, 141, 178, 180, 182, 183, 185,
       187, 188, 190, 192, 195, 197, 201, 202, 206, 210, 211, 215, 216,
       217, 219, 221, 223, 224, 225, 229, 230, 234, 235, 237, 247, 276,
       282, 298, 308, 309, 310, 318, 319, 323, 329, 365, 381, 408, 414,
       418, 428, 429, 432, 436, 437, 444, 449, 450, 451, 453, 454, 465,
       467, 468, 471, 473, 474, 480, 506, 509, 510, 514, 515, 516, 531,
       647, 650, 651, 654, 657, 665, 669, 671, 673, 674, 676, 678, 679,
       683, 685, 688, 689, 690, 692, 694, 697, 699, 700], dtype=int64)

In [16]:
a2xy[:, 73]

array([             nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan,

In [64]:
"""
If 'filtered' is activated, this block will filter out any particles that were manually chosen to be filtered out. I 
chose to filter these out using masked arrays rather than shortening the dictionaries.
"""

#Manually mask particle trajectories here that seem out of whack.

if filtered == True:
    for num in range(0, tofilt.shape[0]):
        M1x[tofilt[num]] = ma.masked_greater(M1x[tofilt[num]], 0)
        M1y[tofilt[num]] = ma.masked_greater(M1y[tofilt[num]], 0)
        M1z[tofilt[num]] = ma.masked_greater(M1z[tofilt[num]], 0)
        M2xy[tofilt[num]] = ma.masked_greater(M2xy[tofilt[num]], 0)
        M2xz[tofilt[num]] = ma.masked_greater(M2xz[tofilt[num]], 0)
        M2yz[tofilt[num]] = ma.masked_greater(M2yz[tofilt[num]], 0)
        M3[tofilt[num]] = ma.masked_greater(M3[tofilt[num]], 0)
    
        SM1x[tofilt[num]] = ma.masked_greater(SM1x[tofilt[num]], 0)
        SM1y[tofilt[num]] = ma.masked_greater(SM1y[tofilt[num]], 0)
        SM1z[tofilt[num]] = ma.masked_greater(SM1z[tofilt[num]], 0)
        SM2xy[tofilt[num]] = ma.masked_greater(SM2xy[tofilt[num]], 0)
        SM2xz[tofilt[num]] = ma.masked_greater(SM2xz[tofilt[num]], 0)
        SM2yz[tofilt[num]] = ma.masked_greater(SM2yz[tofilt[num]], 0)
        SM3[tofilt[num]] = ma.masked_greater(SM3[tofilt[num]], 0)
    
        SD1x[tofilt[num]] = ma.masked_greater(SD1x[tofilt[num]], 0)
        SD1y[tofilt[num]] = ma.masked_greater(SD1y[tofilt[num]], 0)
        SD1z[tofilt[num]] = ma.masked_greater(SD1z[tofilt[num]], 0)
        SD2xy[tofilt[num]] = ma.masked_greater(SD2xy[tofilt[num]], 0)
        SD2xz[tofilt[num]] = ma.masked_greater(SD2xz[tofilt[num]], 0)
        SD2yz[tofilt[num]] = ma.masked_greater(SD2yz[tofilt[num]], 0)
        SD3[tofilt[num]] = ma.masked_greater(SD3[tofilt[num]], 0)
        
        xs[tofilt[num]] = ma.masked_greater(xs[tofilt[num]], 0)
        ys[tofilt[num]] = ma.masked_greater(ys[tofilt[num]], 0)
        zs[tofilt[num]] = ma.masked_greater(zs[tofilt[num]], 0)

In [65]:
"""
OUTDATED.  This block calculates particle totals after filtering out manually filtered particles.
"""

count1 = 0

for num in range(1, fifties +1):
    if ma.count_masked(M1x[num]) > np.shape(M1x[num])[0] - 3:
        count1 = count1 + 1

tot_postfilt = fifties - count1
print('total particles after manual filter:', tot_postfilt)

total particles after manual filter: 589


In [19]:
"""
Old testing code.  Not sure what it was for.
"""

# #np.count_nonzero(M1x[495])
# shift = next((i for i, x in enumerate(M1x[50]) if x), None)
# sM1x = dict()
# sM1x[50] = np.zeros(frames.shape[0] - shift)
# for num in range(0, M1x[50].shape[0] - shift):
#     sM1x[50][num] = M1x[50][num + shift]
# sM1x

'\nOld testing code.  Not sure what it was for.\n'

In [67]:
"""
This block creates a very useful variable, 'yes,' that's a dictionary.  It contains all particles in a given frame.
I use this variable in future blocks to construct new dictionaries of MSD values at each frame.  This is outdated,
once I learned that gmean and mean can operate on 2D arrays.  This means I just need to change my dictionaries to
2D arrays, and averaging can be done much more easily. I will leave it for now.
"""

#Each entry in yes is an array of particles that are in the given frame.  The frame is given by the index of yes.
yes = dict()
yessize = dict()
for num1 in range(1, frames.shape[0]+1):
    count = 0
    yes[num1] = np.zeros(total1).astype(np.int64)
    for num in range(1, total1):
        check = num1 in I[num]
        if check == True:
            yes[num1][count] = num
            count = count + 1
    iindex = np.where(yes[num1]==max(yes[num1]))[0][0]
    yes[num1] = yes[num1][0:iindex]

In [68]:
"""
Implementation of frame-by-frame analysis.  Constructs new dictionaries (f-MSDs) that contain MSD datapoints of all
particles at each frame (key is the frame, instead of the particle number).  These can be used to perform gmean and 
mean functions on 1D arrays.
"""

#And the FBF analyis
framed = dict()
fM1y = dict()
fM1z = dict()
fM2xy = dict()
fM2xz = dict()
fM2yz = dict()
fM3 = dict()

#Due to some issues with the first frame being a NaN, my for loop is cut one short.  Get funky results otherwise.
for num2 in range(1, frames.shape[0]):
    framed[num2] = np.zeros(frames.shape[0])
    fM1y[num2] = np.zeros(frames.shape[0])
    fM1z[num2] = np.zeros(frames.shape[0])
    fM2xy[num2] = np.zeros(frames.shape[0])
    fM2xz[num2] = np.zeros(frames.shape[0])
    fM2yz[num2] = np.zeros(frames.shape[0])
    fM3[num2] = np.zeros(frames.shape[0])
    
    for num1 in range(0, frames.shape[0]):
        frame = np.zeros(yes[num2].shape[0])
        fmM1y = np.zeros(yes[num2].shape[0])
        fmM1z = np.zeros(yes[num2].shape[0])
        fmM2xy = np.zeros(yes[num2].shape[0])
        fmM2xz = np.zeros(yes[num2].shape[0])
        fmM2yz = np.zeros(yes[num2].shape[0])
        fmM3 = np.zeros(yes[num2].shape[0])
        
        for num in range(0, yes[num2].shape[0]):
            frame[num] = M1x[yes[num2][num]][num1]
            fmM1y[num] = M1y[yes[num2][num]][num1]
            fmM1z[num] = M1z[yes[num2][num]][num1]
            fmM2xy[num] = M2xy[yes[num2][num]][num1]
            fmM2xz[num] = M2xz[yes[num2][num]][num1]
            fmM2yz[num] = M2yz[yes[num2][num]][num1]
            fmM3[num] = M3[yes[num2][num]][num1]
        frame = ma.masked_invalid(frame)
        fmM1y = ma.masked_invalid(fmM1y)
        fmM1z = ma.masked_invalid(fmM1z)
        fmM2xy = ma.masked_invalid(fmM2xy)
        fmM2xz = ma.masked_invalid(fmM2xz)
        fmM2yz = ma.masked_invalid(fmM2yz)
        fmM3 = ma.masked_invalid(fmM3)
        
        framed[num2][num1] = stat.gmean(frame)
        fM1y[num2][num1] = stat.gmean(fmM1y)
        fM1z[num2][num1] = stat.gmean(fmM1z)
        fM2xy[num2][num1] = stat.gmean(fmM2xy)
        fM2xz[num2][num1] = stat.gmean(fmM2xz)
        fM2yz[num2][num1] = stat.gmean(fmM2yz)
        fM3[num2][num1] = stat.gmean(fmM3)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [69]:
#Old way of calculating the FBF averages.

# FM1x = np.zeros(frames.shape[0])
# FM1y = np.zeros(frames.shape[0])
# FM1z = np.zeros(frames.shape[0])
# FM2xy = np.zeros(frames.shape[0])
# FM2xz = np.zeros(frames.shape[0])
# FM2yz = np.zeros(frames.shape[0])
# FM3 = np.zeros(frames.shape[0])

# for num in range(1, frames.shape[0]):
#     FM1x = FM1x + framed[num]
#     FM1y = FM1y + fM1y[num]
#     FM1z = FM1z + fM1z[num]
#     FM2xy = FM2xy + fM2xy[num]
#     FM2xz = FM2xz + fM2xz[num]
#     FM2yz = FM2yz + fM2yz[num]
#     FM3 = FM3 + fM3[num]
    
# FM1x = FM1x/frames.shape[0]
# FM1y = FM1y/frames.shape[0]
# FM1z = FM1z/frames.shape[0]
# FM2xy = FM2xy/frames.shape[0]
# FM2xz = FM2xz/frames.shape[0]
# FM2yz = FM2yz/frames.shape[0]
# FM3 = FM3/frames.shape[0]



In [70]:
FM1x = np.zeros(frames.shape[0])
FM1y = np.zeros(frames.shape[0])
FM1z = np.zeros(frames.shape[0])
FM2xy = np.zeros(frames.shape[0])
FM2xz = np.zeros(frames.shape[0])
FM2yz = np.zeros(frames.shape[0])
FM3 = np.zeros(frames.shape[0])

st_FM1x = np.zeros(frames.shape[0])
st_FM1y = np.zeros(frames.shape[0])
st_FM1z = np.zeros(frames.shape[0])
st_FM2xy = np.zeros(frames.shape[0])
st_FM2xz = np.zeros(frames.shape[0])
st_FM2yz = np.zeros(frames.shape[0])
st_FM3 = np.zeros(frames.shape[0])

swframed = dict()
swfM1y = dict()
swfM1z = dict()
swfM2xy = dict()
swfM2xz = dict()
swfM2yz = dict()
swfM3 = dict()

for num in range(1, frames.shape[0]+1):
    swframed[num] = np.zeros(frames.shape[0]-1)
    swfM1y[num] = np.zeros(frames.shape[0]-1)
    swfM1z[num] = np.zeros(frames.shape[0]-1)
    swfM2xy[num] = np.zeros(frames.shape[0]-1)
    swfM2xz[num] = np.zeros(frames.shape[0]-1)
    swfM2yz[num] = np.zeros(frames.shape[0]-1)
    swfM3[num] = np.zeros(frames.shape[0]-1)
    
    for num1 in range(1, frames.shape[0]):
        swframed[num][num1-1] = framed[num1][num-1]
        swfM1y[num][num1-1] = fM1y[num1][num-1]
        swfM1z[num][num1-1] = fM1z[num1][num-1]
        swfM2xy[num][num1-1] = fM2xy[num1][num-1]
        swfM2xz[num][num1-1] = fM2xz[num1][num-1]
        swfM2yz[num][num1-1] = fM2yz[num1][num-1]
        swfM3[num][num1-1] = fM3[num1][num-1]

for num in range(1, frames.shape[0]+1):
    FM1x[num-1] = np.mean(swframed[num])
    FM1y[num-1] = np.mean(swfM1y[num])
    FM1z[num-1] = np.mean(swfM1z[num])
    FM2xy[num-1] = np.mean(swfM2xy[num])
    FM2xz[num-1] = np.mean(swfM2xz[num])
    FM2yz[num-1] = np.mean(swfM2yz[num])
    FM3[num-1] = np.mean(swfM3[num])

    st_FM1x[num-1] = np.std(swframed[num])
    st_FM1y[num-1] = np.std(swfM1y[num])
    st_FM1z[num-1] = np.std(swfM1z[num])
    st_FM2xy[num-1] = np.std(swfM2xy[num])
    st_FM2xz[num-1] = np.std(swfM2xz[num])
    st_FM2yz[num-1] = np.std(swfM2yz[num])
    st_FM3[num-1] = np.std(swfM3[num])

In [71]:
#ty is just a check to see if the size of the resulting arrays was correct.

ty = dict()

for num in range(1, frames.shape[0]):
    ty[num] = np.zeros(frames.shape[0]-1)
    
    for num1 in range(1, frames.shape[0]):
        ty[num][num1-1] = framed[num1][num-1]

# ty[45].shape

In [72]:
# swframed[46] 
# A check to see how swframed was working.

In [73]:
"""
Diffusion coefficients, as calculated by frame-by-frame analysis.  Note that individual diffusion coefficients are
meaningless here, as Ds are calculated based on an average MSD.
"""

FD1x = FM1x/time
FD1y = FM1y/time
FD1z = FM1z/time
FD2xy = FM2xy/time
FD2xz = FM2xz/time
FD2yz = FM2yz/time
FD3 = FM3/time

In [74]:
def plot_MLOG(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
    #I will need to calculate standard deviations eventually
#     #Now to calculate the standard dev at each point:
#     for num in range (1, total1):
#         SDunit = (MSD[num] - MMSD)**2
#         SD = SD + SDunit
#     SD = np.sqrt(SD/total1)
#     SE = SD/np.sqrt(total1)
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    
#     ax.plot(t, linel[0], linewidth=3, color='blue')
#     ax.plot(t, linel[1], linewidth=3, color='red')
#     ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.61), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([0.1, limit1])
    plt.gca().set_ylim([0.0001, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [75]:
"""
Plot of 2D, 1x, and 1y MSDs on a log scale.
"""

one1, bleh = plot_MLOG(FM2xy, FM1x, FM1y, time, 1, 1, 'MSD (um^2)', logplot, 100.1, 0.1, 1, 1)
plt.show()

In [76]:
def plot_M(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.61), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}e'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [77]:
"""
Plots the 2D, 1x, and 1y MSDs on a normal scale.
"""

one1, bleh = plot_M(FM2xy, FM1x, FM1y, time, 0, 1, 'MSD (um^2)', Mplot, 50.1, 0.0241, 10, 0.004)
plt.show()

In [78]:
Deff3 = Deff2*np.ones(time.shape[0])

In [79]:
def plot_M1(M2xy, Mx, My, Mth, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linewidth=10, label='2D', color='blue')
    ax.plot(t, Mx, linewidth=10, label='1D x', color='red')
    ax.plot(t, My, linewidth=10, label='1D x', color='green')
    ax.plot(t, Mth, '--', linewidth=10, label='theoretical', color='purple')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)

    hfont = {'fontname':'Arial'}
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'D ($\mu$m$^2$/s)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.10, 0.54), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}e'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [80]:
"""
Plots the 2D, 1x, and 1y diffusion coefficients on a normal scale.
"""

one1, bleh = plot_M1(FD2xy, FD1x, FD1y, Deff3, time, 0, 1, 'MSD (um^2)', Dplot, 50.1, 0.0021, 10, 0.0004)
plt.show()

In [81]:
"""
This block creates arrays to hold the arithmetic and geometric MSD data.
"""

#Now to calculate arithmetic and geometric means.

arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM1z = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])
arM2xz = np.zeros(SM1x[1].shape[0])
arM2yz = np.zeros(SM1x[1].shape[0])
arM3 = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD1z = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])
arD2xz = np.zeros(SM1x[1].shape[0])
arD2yz = np.zeros(SM1x[1].shape[0])
arD3 = np.zeros(SM1x[1].shape[0])

st_arM1x = np.zeros(SM1x[1].shape[0])
st_arM1y = np.zeros(SM1x[1].shape[0])
st_arM1z = np.zeros(SM1x[1].shape[0])
st_arM2xy = np.zeros(SM1x[1].shape[0])
st_arM2xz = np.zeros(SM1x[1].shape[0])
st_arM2yz = np.zeros(SM1x[1].shape[0])
st_arM3 = np.zeros(SM1x[1].shape[0])

st_arD1x = np.zeros(SM1x[1].shape[0])
st_arD1y = np.zeros(SM1x[1].shape[0])
st_arD1z = np.zeros(SM1x[1].shape[0])
st_arD2xy = np.zeros(SM1x[1].shape[0])
st_arD2xz = np.zeros(SM1x[1].shape[0])
st_arD2yz = np.zeros(SM1x[1].shape[0])
st_arD3 = np.zeros(SM1x[1].shape[0])

In [82]:
"""
This block calculates the arithmetic and geometric MSDs using the clunky define-new-array-by-frame method to do so.
"""
tots = total1 - 1
time2 = time

gM1x = dict()
gM1y = dict()
gM1z = dict()
gM2xy = dict()
gM2xz = dict()
gM2yz = dict()
gM3 = dict()

log_gM1x = dict()
log_gM1y = dict()
log_gM1z = dict()
log_gM2xy = dict()
log_gM2xz = dict()
log_gM2yz = dict()
log_gM3 = dict()

# gD1x = dict()
# gD1y = dict()
# gD1z = dict()
# gD2xy = dict()
# gD2xz = dict()
# gD2yz = dict()
# gD3 = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM1z = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])
geoM2xz = np.zeros(SM1x[1].shape[0])
geoM2yz = np.zeros(SM1x[1].shape[0])
geoM3 = np.zeros(SM1x[1].shape[0])

st_geoM1x = np.zeros(SM1x[1].shape[0])
st_geoM1y = np.zeros(SM1x[1].shape[0])
st_geoM1z = np.zeros(SM1x[1].shape[0])
st_geoM2xy = np.zeros(SM1x[1].shape[0])
st_geoM2xz = np.zeros(SM1x[1].shape[0])
st_geoM2yz = np.zeros(SM1x[1].shape[0])
st_geoM3 = np.zeros(SM1x[1].shape[0])

geoD1x = np.zeros(SM1x[1].shape[0])
geoD1y = np.zeros(SM1x[1].shape[0])
geoD1z = np.zeros(SM1x[1].shape[0])
geoD2xy = np.zeros(SM1x[1].shape[0])
geoD2xz = np.zeros(SM1x[1].shape[0])
geoD2yz = np.zeros(SM1x[1].shape[0])
geoD3 = np.zeros(SM1x[1].shape[0])

st_geoD1x = np.zeros(SM1x[1].shape[0])
st_geoD1y = np.zeros(SM1x[1].shape[0])
st_geoD1z = np.zeros(SM1x[1].shape[0])
st_geoD2xy = np.zeros(SM1x[1].shape[0])
st_geoD2xz = np.zeros(SM1x[1].shape[0])
st_geoD2yz = np.zeros(SM1x[1].shape[0])
st_geoD3 = np.zeros(SM1x[1].shape[0])

for num2 in range(0, SM1x[1].shape[0]):
    gM1x[num2+1] = np.zeros(tots)
    gM1y[num2+1] = np.zeros(tots)
    gM1z[num2+1] = np.zeros(tots)
    gM2xy[num2+1] = np.zeros(tots)
    gM2xz[num2+1] = np.zeros(tots)
    gM2yz[num2+1] = np.zeros(tots)
    gM3[num2+1] = np.zeros(tots)
    
#     gD1x[num2+1] = np.zeros(tots)
#     gD1y[num2+1] = np.zeros(tots)
#     gD1z[num2+1] = np.zeros(tots)
#     gD2xy[num2+1] = np.zeros(tots)
#     gD2xz[num2+1] = np.zeros(tots)
#     gD2yz[num2+1] = np.zeros(tots)
#     gD3[num2+1] = np.zeros(tots)
    
    for num in range(1, tots + 1):
        gM1x[num2+1][num-1] = SM1x[num][num2]
        gM1y[num2+1][num-1] = SM1y[num][num2]
        gM1z[num2+1][num-1] = SM1z[num][num2]
        gM2xy[num2+1][num-1] = SM2xy[num][num2]
        gM2xz[num2+1][num-1] = SM2xz[num][num2]
        gM2yz[num2+1][num-1] = SM2yz[num][num2]
        gM3[num2+1][num-1] = SM3[num][num2]
        
#         gD1x[num2+1][num-1] = SD1x[num][num2]
#         gD1y[num2+1][num-1] = SD1y[num][num2]
#         gD1z[num2+1][num-1] = SD1z[num][num2]
#         gD2xy[num2+1][num-1] = SD2xy[num][num2]
#         gD2xz[num2+1][num-1] = SD2xz[num][num2]
#         gD2yz[num2+1][num-1] = SD2yz[num][num2]
#         gD3[num2+1][num-1] = SD3[num][num2]
        
    gM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
    gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
    gM1z[num2+1] = ma.masked_invalid(gM1z[num2+1])
    gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])
    gM2xz[num2+1] = ma.masked_invalid(gM2xz[num2+1])
    gM2yz[num2+1] = ma.masked_invalid(gM2yz[num2+1])
    gM3[num2+1] = ma.masked_invalid(gM3[num2+1])

    #I need to include this, because geometric means can't be calculated on values of 0.
    gM1x[num2+1] = ma.masked_equal(gM1x[num2+1], 0)
    gM1y[num2+1] = ma.masked_equal(gM1y[num2+1], 0)
    gM1z[num2+1] = ma.masked_equal(gM1z[num2+1], 0)
    gM2xy[num2+1] = ma.masked_equal(gM2xy[num2+1], 0)
    gM2xz[num2+1] = ma.masked_equal(gM2xz[num2+1], 0)
    gM2yz[num2+1] = ma.masked_equal(gM2yz[num2+1], 0)
    gM3[num2+1] = ma.masked_equal(gM3[num2+1], 0)

    log_gM1x[num2+1] = np.log(gM1x[num2+1])
    log_gM1y[num2+1] = np.log(gM1y[num2+1])
    log_gM1z[num2+1] = np.log(gM1z[num2+1])
    log_gM2xy[num2+1] = np.log(gM2xy[num2+1])
    log_gM2xz[num2+1] = np.log(gM2xz[num2+1])
    log_gM2yz[num2+1] = np.log(gM2yz[num2+1])
    log_gM3[num2+1] = np.log(gM3[num2+1])
        
#     gD1x[num2+1] = ma.masked_invalid(SD1x[num2+1])
#     gD1y[num2+1] = ma.masked_invalid(SD1y[num2+1])
#     gD1z[num2+1] = ma.masked_invalid(SD1z[num2+1])
#     gD2xy[num2+1] = ma.masked_invalid(SD2xy[num2+1])
#     gD2xz[num2+1] = ma.masked_invalid(SD2xz[num2+1])
#     gD2yz[num2+1] = ma.masked_invalid(SD2yz[num2+1])
#     gD3[num2+1] = ma.masked_invalid(SD3[num2+1])
        
    geoM1x[num2] = stat.gmean(gM1x[num2+1])
    geoM1y[num2] = stat.gmean(gM1y[num2+1])
    geoM1z[num2] = stat.gmean(gM1z[num2+1])
    geoM2xy[num2] = stat.gmean(gM2xy[num2+1])
    geoM2xz[num2] = stat.gmean(gM2xz[num2+1])
    geoM2yz[num2] = stat.gmean(gM2yz[num2+1])
    geoM3[num2] = stat.gmean(gM3[num2+1])
    
    #(Wiki method) The geometric standard deviation is a function of the mean of the logarithms.
#     st_geoM1x[num2] = np.exp(np.sqrt(np.mean((log_gM1x[num2+1] - np.log(geoM1x[num2]))*(log_gM1x[num2+1] - np.log(geoM1x[num2])))))
#     st_geoM1y[num2] = np.exp(np.sqrt(np.mean((log_gM1y[num2+1] - np.log(geoM1y[num2]))*(log_gM1y[num2+1] - np.log(geoM1y[num2])))))
#     st_geoM1z[num2] = np.exp(np.sqrt(np.mean((log_gM1z[num2+1] - np.log(geoM1z[num2]))*(log_gM1z[num2+1] - np.log(geoM1z[num2])))))
#     st_geoM2xy[num2] = np.exp(np.sqrt(np.mean((log_gM2xy[num2+1] - np.log(geoM2xy[num2]))*(log_gM2xy[num2+1] - np.log(geoM2xy[num2])))))
#     st_geoM2xz[num2] = np.exp(np.sqrt(np.mean((log_gM2xz[num2+1] - np.log(geoM2xz[num2]))*(log_gM2xz[num2+1] - np.log(geoM2xz[num2])))))
#     st_geoM2yz[num2] = np.exp(np.sqrt(np.mean((log_gM2yz[num2+1] - np.log(geoM2yz[num2]))*(log_gM2yz[num2+1] - np.log(geoM2yz[num2])))))
#     st_geoM3[num2] = np.exp(np.sqrt(np.mean((log_gM3[num2+1] - np.log(geoM3[num2]))*(log_gM3[num2+1] - np.log(geoM3[num2])))))

    #Nance method for standard error or geometric mean.
    st_geoM1x[num2] = np.abs(geoM1x[num2]-np.exp(np.mean(np.log(gM1x[num2+1]))-np.std(np.log(gM1x[num2+1]))/np.sqrt(gM1x[num2+1].shape[0])))
    st_geoM1y[num2] = np.abs(geoM1y[num2]-np.exp(np.mean(np.log(gM1y[num2+1]))-np.std(np.log(gM1y[num2+1]))/np.sqrt(gM1y[num2+1].shape[0])))
    st_geoM1z[num2] = np.abs(geoM1z[num2]-np.exp(np.mean(np.log(gM1z[num2+1]))-np.std(np.log(gM1z[num2+1]))/np.sqrt(gM1z[num2+1].shape[0])))
    st_geoM2xy[num2] = np.abs(geoM2xy[num2]-np.exp(np.mean(np.log(gM2xy[num2+1]))-np.std(np.log(gM2xy[num2+1]))/np.sqrt(gM2xy[num2+1].shape[0])))
    st_geoM2xz[num2] = np.abs(geoM2xz[num2]-np.exp(np.mean(np.log(gM2xz[num2+1]))-np.std(np.log(gM2xz[num2+1]))/np.sqrt(gM2xz[num2+1].shape[0])))
    st_geoM2yz[num2] = np.abs(geoM2yz[num2]-np.exp(np.mean(np.log(gM2yz[num2+1]))-np.std(np.log(gM2yz[num2+1]))/np.sqrt(gM2yz[num2+1].shape[0])))
    st_geoM3[num2] = np.abs(geoM3[num2]-np.exp(np.mean(np.log(gM3[num2+1]))-np.std(np.log(gM3[num2+1]))/np.sqrt(gM3[num2+1].shape[0])))

#     geoD1x[num2] = stat.gmean(gD1x[num2+1])
#     geoD1y[num2] = stat.gmean(gD1y[num2+1])
#     geoD1z[num2] = stat.gmean(gD1z[num2+1])
#     geoD2xy[num2] = stat.gmean(gD2xy[num2+1])
#     geoD2xz[num2] = stat.gmean(gD2xz[num2+1])
#     geoD2yz[num2] = stat.gmean(gD2yz[num2+1])
#     geoD3[num2] = stat.gmean(gD3[num2+1])

    geoD1x[num2] = geoM1x[num2]/time2[num2]
    geoD1y[num2] = geoM1y[num2]/time2[num2]
    geoD1z[num2] = geoM1z[num2]/time2[num2]
    geoD2xy[num2] = geoM2xy[num2]/time2[num2]
    geoD2xz[num2] = geoM2xz[num2]/time2[num2]
    geoD2yz[num2] = geoM2yz[num2]/time2[num2]
    geoD3[num2] = geoM3[num2]/time2[num2]

    arM1x[num2] = np.mean(gM1x[num2+1])
    arM1y[num2] = np.mean(gM1y[num2+1])
    arM1z[num2] = np.mean(gM1z[num2+1])
    arM2xy[num2] = np.mean(gM2xy[num2+1])
    arM2xz[num2] = np.mean(gM2xz[num2+1])
    arM2yz[num2] = np.mean(gM2yz[num2+1])
    arM3[num2] = np.mean(gM3[num2+1])
    
    st_arM1x[num2] = np.std(gM1x[num2+1])
    st_arM1y[num2] = np.std(gM1y[num2+1])
    st_arM1z[num2] = np.std(gM1z[num2+1])
    st_arM2xy[num2] = np.std(gM2xy[num2+1])
    st_arM2xz[num2] = np.std(gM2xz[num2+1])
    st_arM2yz[num2] = np.std(gM2yz[num2+1])
    st_arM3[num2] = np.std(gM3[num2+1])


#     arD1x[num2] = np.mean(gD1x[num2+1])
#     arD1y[num2] = np.mean(gD1y[num2+1])
#     arD1z[num2] = np.mean(gD1z[num2+1])
#     arD2xy[num2] = np.mean(gD2xy[num2+1])
#     arD2xz[num2] = np.mean(gD2xz[num2+1])
#     arD2yz[num2] = np.mean(gD2yz[num2+1])
#     arD3[num2] = np.mean(gD3[num2+1])

    arD1x[num2] = arM1x[num2]/time2[num2]
    arD1y[num2] = arM1y[num2]/time2[num2]
    arD1z[num2] = arM1z[num2]/time2[num2]
    arD2xy[num2] = arM2xy[num2]/time2[num2]
    arD2xz[num2] = arM2xz[num2]/time2[num2]
    arD2yz[num2] = arM2yz[num2]/time2[num2]
    arD3[num2] = arM3[num2]/time2[num2]

    st_arD1x[num2] = st_arM1x[num2]/time2[num2]
    st_arD1y[num2] = st_arM1y[num2]/time2[num2]
    st_arD1z[num2] = st_arM1z[num2]/time2[num2]
    st_arD2xy[num2] = st_arM2xy[num2]/time2[num2]
    st_arD2xz[num2] = st_arM2xz[num2]/time2[num2]
    st_arD2yz[num2] = st_arM2yz[num2]/time2[num2]
    st_arD3[num2] = st_arM3[num2]/time2[num2]

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [83]:
def plot_Mcomp(Mgeo, Mar, MFBF, t1, t2, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Mgeo = Geometric average dataset
    Mar = Arithmetic average dataset
    MFBF = Frame-by-Frame dataset
    
    t1 = time array for Mgeo and Mar
    t2 time array for MFDF
    """
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t1, Mgeo, linewidth=10, label='Geometric', color='blue')
    ax.plot(t1, Mar, linewidth=10, label='Arithmetic', color='red')
    ax.plot(t2, MFBF, linewidth=10, label='Frame-by-Frame', color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.60, 0.66), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return Mar, Mgeo

In [84]:
"""
This block compares MSDs calculated by frame-by-frame, geometric, and arithmetic means.
"""

one1, bleh = plot_Mcomp(geoM2xy, arM2xy, FM2xy, time2, time, 0, 2, 'MSD (um^2)', Cplot, 50.1, 0.051, 10, 0.01)
plt.show()

In [85]:
"""
This block allows the user to manually check the calculated arithmetic mean.
"""
np.set_printoptions(precision=5, suppress=True)
np.round(arM2xy, 5)

array([     nan,  0.12402,  0.20542,  0.26681,  0.34416,  0.06211,
        0.06752,  0.06511,  0.05929,  0.05741,  0.05845,  0.06023,
        0.06167,  0.06228,  0.06091,  0.05853,  0.06033,  0.06142,
        0.06113,  0.0612 ,  0.06093,  0.06239,  0.06442,  0.06296,
        0.06312,  0.06381,  0.06331,  0.06387,  0.06013,  0.06474,
        0.06759,  0.0727 ,  0.06133,  0.06119,  0.0628 ,  0.06309,
        0.06444,  0.06733,  0.06059,  0.06106,  0.06215,  0.06322,
        0.06036,  0.0605 ,  0.0628 ,  0.06385,  0.06692,  0.06471,
        0.06653,  0.06858,  0.0692 ,  0.0688 ,  0.07052,  0.07217,
        0.0733 ,  0.07911,  0.07066,  0.07274,  0.07193,  0.07083,
        0.07107,  0.07223,  0.07211,  0.07197,  0.07195,  0.07705,
        0.07486,  0.07729,  0.07585,  0.07508,  0.07395,  0.0653 ,
        0.05486,  0.06091])

In [86]:
"""
This block allows the user to manually check the calculated geometric mean.
"""
np.round(geoM2xy, 5)

array([     nan,  0.01624,  0.01761,  0.01772,  0.0176 ,  0.01483,
        0.01505,  0.01501,  0.0151 ,  0.01489,  0.01462,  0.01499,
        0.01525,  0.01552,  0.01526,  0.01544,  0.0157 ,  0.01591,
        0.01603,  0.0163 ,  0.01662,  0.01679,  0.01718,  0.01705,
        0.01732,  0.01747,  0.01764,  0.01804,  0.01843,  0.01883,
        0.01886,  0.01922,  0.01994,  0.02021,  0.02082,  0.02106,
        0.02153,  0.02191,  0.02195,  0.02235,  0.02285,  0.0232 ,
        0.02342,  0.02399,  0.02443,  0.02449,  0.02486,  0.02533,
        0.02565,  0.02606,  0.02659,  0.02656,  0.02741,  0.02761,
        0.02765,  0.02802,  0.02794,  0.02821,  0.0285 ,  0.02885,
        0.02917,  0.02973,  0.03022,  0.03068,  0.03097,  0.03107,
        0.03167,  0.03138,  0.03051,  0.02989,  0.02783,  0.02514,
        0.01712,  0.0188 ])

In [87]:
"""
This block allows the user to manually check the calculated frame-by-frame mean.
"""
np.round(FM2xy, 5)

array([     nan,  0.01115,  0.01149,  0.01178,  0.01184,  0.01206,
        0.01226,  0.01232,  0.01256,  0.01283,  0.01279,  0.0131 ,
        0.01327,  0.01361,  0.01378,  0.01413,  0.01428,  0.01459,
        0.01466,  0.01496,  0.01534,  0.01546,  0.01595,  0.01605,
        0.01635,  0.0167 ,  0.017  ,  0.01754,  0.01773,  0.01814,
        0.01842,  0.01874,  0.01932,  0.01958,  0.02013,  0.02044,
        0.02099,  0.02124,  0.02144,  0.02194,  0.02255,  0.02299,
        0.0234 ,  0.02387,  0.02439,  0.02441,  0.02472,  0.02532,
        0.02585,  0.02637,  0.02672,  0.02688,  0.02767,  0.0279 ,
        0.02807,  0.02838,  0.02871,  0.02903,  0.02929,  0.02964,
        0.02998,  0.03054,  0.03139,  0.03194,  0.03213,  0.03221,
        0.03279,  0.0326 ,  0.03167,  0.03098,  0.02883,  0.02625,
        0.01823,  0.02062])

In [88]:
"""
This block allows the user to check for faulty trajectories.  The user can manipulate the frame at which to check for 
large MSDs and set a threshold at which to check.
"""

check_frame = 3
thresh = 0.4

at_fault = np.where(gM2xy[check_frame]> thresh)[0] + 1
# gM1x[2][34]
# gM1x[2][218]
at_fault
#M2xy[at_fault[1]+1]

array([ 33,  45, 118, 198, 199, 207, 208, 231, 233, 254, 281, 354, 388,
       417, 426, 431, 434, 445, 447, 448, 458, 543, 544, 644, 645, 652,
       655, 663, 664, 667, 677, 684], dtype=int64)

In [89]:
def plot_histogram(M, time, total, datatype, filename, tn):
    """
    This function plots a histogram of the desired input, typically an MSD dataset or a diffusion dataset.
    
    Inputs: 
    M: MSD or Deff dataset
    time: time np array
    total: total number of particles
    datatype:
    filename: 
    tn: 
    """
    total1 = total
    MSD = M

    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]

    hist = [x for x in hist if str(x) != 'nan']
    
    plot, bins = np.histogram(hist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
#     plt.xlabel('D (um2/s)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [90]:
"""
Prints the gemoetric, arithemtic, and frame-by-frame diffusion coefficients at 1 second.
"""

def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return array[idx], idx

time_a, tindex = find_nearest(time2, 1)
geoDat1 = geoD2xy[tindex]
arDat1 = arD2xy[tindex]

time_a1, tindex1 = find_nearest(time, 1)
FDat1 = FD2xy[tindex1]

print('geometric average at 1s:', np.round(geoDat1, 5), 'um2/s with', tot_postfilt, 'particles')
print('arithmetic average at 1s:', np.round(arDat1, 5), 'um2/s with', tot_postfilt, 'particles')
print('frame-by-frame average at 1s:', np.round(FDat1, 5), 'um2/s with', tot_postfilt, 'particles')

geometric average at 1s: 0.01427 um2/s with 589 particles
arithmetic average at 1s: 0.16639 um2/s with 589 particles
frame-by-frame average at 1s: 0.00931 um2/s with 589 particles


In [91]:
"""
Plots a histogram of calculated diffusion coefficients at 1 second.
"""

a, hist, total= plot_histogram(M2xy, time2, tots, 'MSD (um^2)', Hplot, 10)
plt.show()

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [92]:
def plot_loghistogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.
    """

    total1 = total
    MSD = M
    
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]
    
    z1, z2 = find_nearest(hist[1:], 0)
    hist = np.delete(hist[1:], z2)
    hist = [x for x in hist if str(x) != 'nan']
    loghist = np.log(hist)

    plot, bins = np.histogram(loghist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('log(D)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [93]:
"""
Plots a histogram of the logarithm of the diffusion coefficients at 1 second.
"""

hist, MSD, total = plot_loghistogram(M2xy, time2, tots, 'MSD (um^2)', Hlogplot, 5)
plt.show()

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [94]:
"""
This block saves calculated MSD and Deff arrays as csv files to be imported into other notebooks.
"""

np.savetxt('geoD2xy_{}.csv'.format(name), geoD2xy, delimiter=',')
np.savetxt('arD2xy_{}.csv'.format(name), arD2xy, delimiter=',')
np.savetxt('FD2xy_{}.csv'.format(name), FD2xy, delimiter=',')

np.savetxt('geoM2xy_{}.csv'.format(name), geoM2xy, delimiter=',')
np.savetxt('arM2xy_{}.csv'.format(name), arM2xy, delimiter=',')
np.savetxt('FM2xy_{}.csv'.format(name), FM2xy, delimiter=',')

In [95]:
Tplot = '{}_Tplot'.format(name)

In [96]:
"""
This block plots all trajectories without centering.  The user can adjust the range of x and y directly below.
"""

center_zoom = 80 
maxx = 200 - center_zoom
minx = 0 + center_zoom
maxy = 200 - center_zoom
miny = 0 + center_zoom

def plot_traj(xmax, xmin, ymax, ymin, xts, yts):
    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    for num in range(1, total1):
        ax.plot(xts[num], yts[num], linewidth=10, label='Particle {}'.format(num))


    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(xmin, xmax + 0.001, 20)
    ymajor_ticks = np.arange(ymin, ymax + 0.001, 20)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('x (um)', fontsize=95)
    ax.set_ylabel('y (um)', fontsize=95)
    #ax.tick_params(direction='out', pad=16)
    #ax.legend(loc=(0.60, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(0)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(0)))


    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([xmin, xmax])
    plt.gca().set_ylim([ymin, ymax])

    # Save your figure
    plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')

plot_traj(maxx, minx, maxy, miny, xs, ys)
plt.show()

In [124]:
"""
This block plots user-defined trajectories, defined by their particle numbers.  The user can also adjust the window by
using the unit block scaling all axes.
"""

unit = 6
NPs = np.array([1, 10, 20, 30, 40, 50])
#NPs = filte

maxx = 0.1*unit
minx = -0.1*unit
maxy = 0.1*unit
miny = -0.1*unit
ticks = 0.04*unit
dec = 2


#NPs = filte
#NPs = np.array([30])
xc = dict()
yc = dict()
cxmask = dict()

# Creates figure
fig1 = plt.figure(figsize=(24, 18), dpi=80)
ax1 = fig1.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

for num in range(1, NPs.shape[0]+1):
    lowx = ma.min(xs[NPs[num-1]])
    highx = ma.max(xs[NPs[num-1]])
    lowy = ma.min(ys[NPs[num-1]])
    highy = ma.max(ys[NPs[num-1]])
    
    xc[NPs[num-1]] = np.array([xs[NPs[num-1]] - ((highx+lowx)/2)])
    yc[NPs[num-1]] = np.array([ys[NPs[num-1]] - ((highy+lowy)/2)])
    
    xcmask[NPs[num-1]] = xs[NPs[num-1]].recordmask
    xc[NPs[num-1]] = ma.array(xc[NPs[num-1]], mask = xcmask[NPs[num-1]])
    yc[NPs[num-1]] = ma.array(yc[NPs[num-1]], mask = xcmask[NPs[num-1]])
    
    ax1.plot(xc[NPs[num-1]][0, :], yc[NPs[num-1]][0, :], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# A few adjustments to prettify the graph
for item in ([ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(minx, maxx + 0.00001, ticks)
ymajor_ticks = np.arange(miny, maxy + 0.00001, ticks)

ax1.set_xticks(xmajor_ticks)
ax1.set_yticks(ymajor_ticks)
ax1.title.set_fontsize(70)
ax1.set_xlabel('x (um)', fontsize=95)
ax1.set_ylabel('y (um)', fontsize=95)
ax1.tick_params(direction='out', pad=16)
plt.xticks(rotation=-30)
#ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([minx, maxx])
plt.gca().set_ylim([miny, maxy])

# Save your figure
plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
plt.show()

In [121]:
num = 2
xcmask = dict()
lowx = ma.min(xs[NPs[num-1]])
highx = ma.max(xs[NPs[num-1]])
lowy = ma.min(ys[NPs[num-1]])
highy = ma.max(ys[NPs[num-1]])

xc[NPs[num-1]] = np.array([xs[NPs[num-1]] - ((highx+lowx)/2)])
yc[NPs[num-1]] = np.array([ys[NPs[num-1]] - ((highy+lowy)/2)])
xc[NPs[num-1]]
xcmask[NPs[num-1]] = xs[NPs[num-1]].recordmask
xc[NPs[num-1]] = ma.array(xc[NPs[num-1]], mask = xcmask[NPs[num-1]])

xc[10]

masked_array(data =
 [[-- -0.13321499999999276 -0.010374999999996248 0.09005499999999245
  -0.05519499999999766 -0.06183500000000208 0.12159499999999923
  -0.17471500000000617 -0.18550500000000625 -0.03942499999999427
  0.003734999999991828 -0.018675000000001774 0.08009500000000003
  0.08673500000000445 -0.05519499999999766 -0.12989500000000476
  -0.1689050000000094 -0.12989500000000476 -0.23447500000000332
  0.005395000000007144 -0.124085000000008 -0.20044500000000198
  -0.06183500000000208 0.07179500000000871 0.034445000000005166
  0.12325500000000034 -0.14400500000000704 -0.007885000000001696
  0.027804999999986535 -0.0668150000000054 -0.10167500000000018
  -0.11578500000000247 -0.044404999999997585 0.2519050000000078
  0.09752499999999031 0.03942499999999427 -0.02614499999999964
  -0.02946500000000185 -0.09254500000000121 -0.08175500000000113
  -0.16060500000000388 -0.19712499999999977 -0.10831499999999039
  -0.14566499999999394 -0.1514750000000049 -0.06349500000000319
  -0.0734550

In [120]:
NPs[1]


10

In [99]:
M2xy[NPs[2]]

masked_array(data = [-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --],
             mask = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True],
       fill_value = 0.0)

In [100]:
b = np.zeros((frames.shape[0], fifties -1))

for num in range(0, fifties - 1):
    b[:, num] = xs[num+1]

b = ma.masked_equal(b, 0)
b

masked_array(data =
 [[100.02412999999999 123.83101999999998 184.75716999999997 ..., 157.43523
  79.23512 150.83756]
 [99.99757 123.86173 184.70570999999998 ..., 158.09425 79.23512 150.73381]
 [100.00005999999999 123.83101999999998 184.72728999999998 ...,
  158.92839999999998 79.19943 150.73381]
 ..., 
 [99.96021999999999 123.7447 184.58867999999998 ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]],
             mask =
 [[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]],
       fill_value = 0.0)

In [101]:
b[(b > 0) & (b < 0.6)]
np.where(b == 0.5917899999999999)
b[1, 595]

42.797289999999997

In [55]:
xs[596]

masked_array(data = [-- 42.79729 42.80227 42.81887 42.8529 42.84128 42.889419999999994 42.81472
 42.83132 42.79231 42.79231 42.84128 42.80144 42.933409999999995 42.79065
 42.87863 42.80144 42.84626 42.93175 42.824679999999994 42.80144 42.76243
 42.80227 42.881119999999996 42.88361 42.84045 42.87697 42.83796 42.83547
 42.827169999999995 42.77654 42.784009999999995 42.750809999999994
 42.803929999999994 42.85456 42.87779999999999 42.78816 42.83962 42.89938
 42.74666 42.76907 42.74417 42.77737 42.742509999999996 42.753299999999996
 42.77239 42.77737 42.793969999999995 42.79231 42.77654 42.73172
 42.697689999999994 42.76409 42.8114 42.75164 42.68109 42.730889999999995
 42.71927 42.74002 42.7367 42.740849999999995 42.80144 42.75662 42.73338
 42.69105 42.785669999999996 42.700179999999996 42.78733 42.74749
 42.760769999999994 42.697689999999994 42.745 42.72923 --],
             mask = [ True False False False False False False False False False False False
 False False False False False Fals

In [56]:
"""
IMPLEMENTED IN ENTIRE CODE.

This code is still experimental, and will be implemented throughout the code eventually.  I have been calculating
MSDs without taking into account lag time, or without a good understanding of it.  I calculated the MSD using a single
start point, rather than adjusting it and averaging it at each value.  This block calculates MSDs with lag time
accounted for, taking averages.

This uses just a single particle.  It doesn't average all particles.  It also only calculates 1D x MSDs.
"""

#Next, I will make a code to calculate MSD the right way-- as a function of lag time instead of just time.
#This block can calculate MSD(tau) for a single particle chosen by the user.

# yourNP = 20

# m1x = np.zeros(frames.shape[0])
# m1y = np.zeros(frames.shape[0])
# m1z = np.zeros(frames.shape[0])
# m2xy = np.zeros(frames.shape[0])
# m2xz = np.zeros(frames.shape[0])
# m2yz = np.zeros(frames.shape[0])
# m3 = np.zeros(frames.shape[0])

# m1xa = dict()
# m1ya = dict()
# m1za = dict()
# m2xya = dict()
# m2xza = dict()
# m2yza = dict()
# m3a = dict()

# for num1 in range(1, frames.shape[0]):

#     tau = num1

#     m1xa[num1] = np.zeros(frames.shape[0] - tau)
#     m1ya[num1] = np.zeros(frames.shape[0] - tau)
#     m1za[num1] = np.zeros(frames.shape[0] - tau)
#     m2xya[num1] = np.zeros(frames.shape[0] - tau)
#     m2xza[num1] = np.zeros(frames.shape[0] - tau)
#     m2yza[num1] = np.zeros(frames.shape[0] - tau)
#     m3a[num1] = np.zeros(frames.shape[0] - tau)
    
#     for num in range(0, frames.shape[0] - tau):
#         m1xa[num1][num] = (xs[yourNP][num + tau] - xs[yourNP][num])**2
#         m1ya[num1][num] = (ys[yourNP][num + tau] - ys[yourNP][num])**2
#         m1za[num1][num] = (zs[yourNP][num + tau] - zs[yourNP][num])**2
#         m2xya[num1][num] = m1xa[num1][num] + m1ya[num1][num]
#         m2xza[num1][num] = m1xa[num1][num] + m1za[num1][num]
#         m2yza[num1][num] = m1za[num1][num] + m1ya[num1][num]
#         m3a[num1][num] = m1xa[num1][num] + m1ya[num1][num] + m1za[num1][num]
    
#     m1xa[num1] = ma.masked_invalid(m1xa[num1])
#     m1ya[num1] = ma.masked_invalid(m1ya[num1])
#     m1za[num1] = ma.masked_invalid(m1za[num1])
#     m2xya[num1] = ma.masked_invalid(m2xya[num1])
#     m2xza[num1] = ma.masked_invalid(m2xza[num1])
#     m2yza[num1] = ma.masked_invalid(m2yza[num1])
#     m3a[num1] = ma.masked_invalid(m3a[num1])
    
#     m1x[num1] = np.mean(m1xa[num1])
#     m1y[num1] = np.mean(m1ya[num1])
#     m1z[num1] = np.mean(m1za[num1])
#     m2xy[num1] = np.mean(m2xya[num1])
#     m2xz[num1] = np.mean(m2xza[num1])
#     m2yz[num1] = np.mean(m2yza[num1])
#     m3[num1] = np.mean(m3a[num1])

# m1x = ma.masked_invalid(m1x)
# m1y = ma.masked_invalid(m1y)
# m1z = ma.masked_invalid(m1z)
# m2xy = ma.masked_invalid(m2xy)
# m2xz = ma.masked_invalid(m2xz)
# m2yz = ma.masked_invalid(m2yz)
# m3 = ma.masked_invalid(m3)

"\nIMPLEMENTED IN ENTIRE CODE.\n\nThis code is still experimental, and will be implemented throughout the code eventually.  I have been calculating\nMSDs without taking into account lag time, or without a good understanding of it.  I calculated the MSD using a single\nstart point, rather than adjusting it and averaging it at each value.  This block calculates MSDs with lag time\naccounted for, taking averages.\n\nThis uses just a single particle.  It doesn't average all particles.  It also only calculates 1D x MSDs.\n"

In [57]:
"""
This block plots the resulting MSDs calculated using time lags for a single particle.
"""

# scale = 'linear' # log or linear options
# NPs = np.array([1, 10, 20, 30, 40, 50])

# #For normal scaling
# if scale == 'linear':
#     maxx = 45
#     minx = 0
#     maxy = 0.1
#     miny = 0
#     xticks = 10
#     yticks = 0.02
#     xdec = 0
#     ydec = 2

# else:
#     maxx = 100
#     minx = 0.1
#     maxy = 1
#     miny = 0.001
#     xticks = 10
#     yticks = 0.02
#     xdec = 0
#     ydec = 2


# #NPs = filte
# #NPs = np.array([30])
# xc = dict()
# yc = dict()

# # Creates figure
# fig1 = plt.figure(figsize=(24, 18), dpi=80)
# ax1 = fig1.add_subplot(111)
# # ax.set_title('Particle Trajectories', x=0.5, y=1.15)
    
# ax1.plot(time[1:], m2xy[1:], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# # A few adjustments to prettify the graph
# for item in ([ax1.xaxis.label, ax1.yaxis.label] +
#              ax1.get_xticklabels() + ax1.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(minx, maxx + 0.00001, xticks)
# ymajor_ticks = np.arange(miny, maxy + 0.00001, yticks)

# ax1.set_xticks(xmajor_ticks)
# ax1.set_yticks(ymajor_ticks)
# ax1.title.set_fontsize(70)
# ax1.set_xlabel('x (um)', fontsize=95)
# ax1.set_ylabel('y (um)', fontsize=95)
# ax1.tick_params(direction='out', pad=16)
# plt.xticks(rotation=-30)
# #ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(xdec)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(ydec)))


# plt.yscale(scale)
# plt.xscale(scale)
# plt.gca().set_xlim([minx, maxx])
# plt.gca().set_ylim([miny, maxy])

# # Save your figure
# plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
# plt.show()

'\nThis block plots the resulting MSDs calculated using time lags for a single particle.\n'

In [58]:
"""
This block takes the functionality from above calculating MSDs and averages over ALL particles.
"""

# am1x = dict()
# am1y = dict()
# am1z = dict()
# am2xy = dict()
# am2xz = dict()
# am2yz = dict()
# am3 = dict()

# m1xa = dict()
# m1ya = dict()
# m1za = dict()
# m2xya = dict()
# m2xza = dict()
# m2yza = dict()
# m3a = dict()

# a1x = np.zeros((total1, frames.shape[0]))
# a1y = np.zeros((total1, frames.shape[0]))
# a1z = np.zeros((total1, frames.shape[0]))
# a2xy = np.zeros((total1, frames.shape[0]))
# a2xz = np.zeros((total1, frames.shape[0]))
# a2yz = np.zeros((total1, frames.shape[0]))
# a3 = np.zeros((total1, frames.shape[0]))

# for num2 in range(1, fifties + 1):

#     am1x[num2] = np.zeros(frames.shape[0])
#     am1y[num2] = np.zeros(frames.shape[0])
#     am1z[num2] = np.zeros(frames.shape[0])
#     am2xy[num2] = np.zeros(frames.shape[0])
#     am2xz[num2] = np.zeros(frames.shape[0])
#     am2yz[num2] = np.zeros(frames.shape[0])
#     am3[num2] = np.zeros(frames.shape[0])

#     for num1 in range(1, frames.shape[0]):

#         tau = num1
        
#         m1xa[num1] = np.zeros(frames.shape[0] - tau)
#         m1ya[num1] = np.zeros(frames.shape[0] - tau)
#         m1za[num1] = np.zeros(frames.shape[0] - tau)
#         m2xya[num1] = np.zeros(frames.shape[0] - tau)
#         m2xza[num1] = np.zeros(frames.shape[0] - tau)
#         m2yza[num1] = np.zeros(frames.shape[0] - tau)
#         m3a[num1] = np.zeros(frames.shape[0] - tau)

#         for num in range(0, frames.shape[0] - tau):
#             m1xa[num1][num] = (xs[num2][num + tau] - xs[num2][num])**2
#             m1ya[num1][num] = (ys[num2][num + tau] - ys[num2][num])**2
#             m1za[num1][num] = (zs[num2][num + tau] - zs[num2][num])**2
#             m2xya[num1][num] = m1xa[num1][num] + m1ya[num1][num]
#             m2xza[num1][num] = m1xa[num1][num] + m1za[num1][num]
#             m2yza[num1][num] = m1za[num1][num] + m1ya[num1][num]
#             m3a[num1][num] = m1xa[num1][num] + m1ya[num1][num] + m1za[num1][num]
            
#         m1xa[num1] = ma.masked_invalid(m1xa[num1])
#         m1ya[num1] = ma.masked_invalid(m1ya[num1])
#         m1za[num1] = ma.masked_invalid(m1za[num1])
#         m2xya[num1] = ma.masked_invalid(m2xya[num1])
#         m2xza[num1] = ma.masked_invalid(m2xza[num1])
#         m2yza[num1] = ma.masked_invalid(m2yza[num1])
#         m3a[num1] = ma.masked_invalid(m3a[num1])
        
#         am1x[num2][num1] = np.mean(m1xa[num1])
#         am1y[num2][num1] = np.mean(m1ya[num1])
#         am1z[num2][num1] = np.mean(m1za[num1])
#         am2xy[num2][num1] = np.mean(m2xya[num1])
#         am2xz[num2][num1] = np.mean(m2xza[num1])
#         am2yz[num2][num1] = np.mean(m2yza[num1])
#         am3[num2][num1] = np.mean(m3a[num1])

#     am1x[num2] = ma.masked_invalid(am1x[num2])
#     am1y[num2] = ma.masked_invalid(am1y[num2])
#     am1z[num2] = ma.masked_invalid(am1z[num2])
#     am2xy[num2] = ma.masked_invalid(am2xy[num2])
#     am2xz[num2] = ma.masked_invalid(am2xz[num2])
#     am2yz[num2] = ma.masked_invalid(am2yz[num2])
#     am3[num2] = ma.masked_invalid(am3[num2])
    
#     a1x[num2-1, :] = am1x[num2]
#     a1y[num2-1, :] = am1y[num2]
#     a1z[num2-1, :] = am1z[num2]
#     a2xy[num2-1, :] = am2xy[num2]
#     a2xz[num2-1, :] = am2xz[num2]
#     a2yz[num2-1, :] = am2yz[num2]
#     a3[num2-1, :] = am3[num2]

# a1x = ma.masked_invalid(a1x)
# a1y = ma.masked_invalid(a1y)
# a1z = ma.masked_invalid(a1z)
# a2xy = ma.masked_invalid(a2xy)
# a2xz = ma.masked_invalid(a2xz)
# a2yz = ma.masked_invalid(a2yz)
# a3 = ma.masked_invalid(a3)

# tot_m1x = np.mean(a1x, axis=0)
# tot_m1y = np.mean(a1y, axis=0)
# tot_m1z = np.mean(a1z, axis=0)
# tot_m2xy = np.mean(a2xy, axis=0)
# tot_m2xz = np.mean(a2xz, axis=0)
# tot_m2yz = np.mean(a2yz, axis=0)
# tot_m3 = np.mean(a3, axis=0)



'\nThis block takes the functionality from above calculating MSDs and averages over ALL particles.\n'

In [59]:
"""
A check to the success of the coding.  Should have dimensions # of particles by # of frames
"""

# a1x[371, :]

'\nA check to the success of the coding.  Should have dimensions # of particles by # of frames\n'

In [60]:
total1

701

In [61]:
"""
This block is a final filter again large datasets.  It checks to see if there are any MSDs greater than a certain
threshold at a given tau.
"""

# thresh1 = 0.5
# given_tau = 5

# filte = np.where(a1[:, given_tau]> thresh1)[0]+1
# filte

'\nThis block is a final filter again large datasets.  It checks to see if there are any MSDs greater than a certain\nthreshold at a given tau.\n'

In [62]:
"""
This block plots the new MSDs taking into account lag times.
"""

# scale = 'linear' # log or linear options
# NPs = np.array([1, 10, 20, 30, 40, 50])

# #For normal scaling
# if scale == 'linear':
#     maxx = 45
#     minx = 0
#     maxy = 0.1
#     miny = 0
#     xticks = 10
#     yticks = 0.02
#     xdec = 0
#     ydec = 2

# else:
#     maxx = 100
#     minx = 0.1
#     maxy = 1
#     miny = 0.001
#     xticks = 10
#     yticks = 0.02
#     xdec = 0
#     ydec = 2

# NPs = np.array([1, 10, 20, 30, 40, 50])
# #NPs = filte
# #NPs = np.array([30])
# xc = dict()
# yc = dict()

# # Creates figure
# fig1 = plt.figure(figsize=(24, 18), dpi=80)
# ax1 = fig1.add_subplot(111)
# # ax.set_title('Particle Trajectories', x=0.5, y=1.15)
    
# ax1.plot(time[1:], tot_m1x[1:], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# # A few adjustments to prettify the graph
# for item in ([ax1.xaxis.label, ax1.yaxis.label] +
#              ax1.get_xticklabels() + ax1.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(minx, maxx + 0.00001, xticks)
# ymajor_ticks = np.arange(miny, maxy + 0.00001, yticks)

# ax1.set_xticks(xmajor_ticks)
# ax1.set_yticks(ymajor_ticks)
# ax1.title.set_fontsize(70)
# ax1.set_xlabel('x (um)', fontsize=95)
# ax1.set_ylabel('y (um)', fontsize=95)
# ax1.tick_params(direction='out', pad=16)
# plt.xticks(rotation=-30)
# #ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(xdec)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(ydec)))


# plt.yscale(scale)
# plt.xscale(scale)
# plt.gca().set_xlim([minx, maxx])
# plt.gca().set_ylim([miny, maxy])

# # Save your figure
# plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
# plt.show()

'\nThis block plots the new MSDs taking into account lag times.\n'